In [30]:
file_name='../results/results_rq1_20201120-182327.pkl'

import pickle
import pandas as pd

results_df=[]
with open(file_name, 'rb') as f:
    results_df = pickle.load(f)
    
def merge_list(list_):
    retVal=''
    for i in list_:
        retVal=' + '.join((retVal, i))
    return retVal[2:]


results_df2=results_df[['train_domain', 'train_domain_modified', 'test_domain', 'test_domain_modified', 'model_name', 'feature_names', 'macro avg f1-score',]]
results_df2['feature_names']=results_df2['feature_names'].apply(lambda x: merge_list(x))
results_df2['train_domain']=results_df2['train_domain'].apply(lambda x: merge_list(x))
results_df2['test_domain']=results_df2['test_domain'].apply(lambda x: merge_list(x))

results_df2.groupby(['train_domain', 'test_domain', 'model_name', 'feature_names'])['macro avg f1-score'].mean().reset_index()

<ipython-input-30-b64feccf5d0f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df2['feature_names']=results_df2['feature_names'].apply(lambda x: merge_list(x))
<ipython-input-30-b64feccf5d0f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df2['train_domain']=results_df2['train_domain'].apply(lambda x: merge_list(x))
<ipython-input-30-b64feccf5d0f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

,train_domain,test_domain,model_name,feature_names,macro avg f1-score
0,benevolent + hostile + other,benevolent + hostile + other,logistic_regression,ngram,0.791033
1,benevolent + hostile + other,benevolent + hostile + other,logistic_regression,ngram + type_dependency,0.776007
2,benevolent + hostile + other,benevolent + hostile + other,logistic_regression,sentiment,0.498996
3,benevolent + hostile + other,benevolent + hostile + other,logistic_regression,sentiment + ngram,0.787287
4,benevolent + hostile + other,benevolent + hostile + other,logistic_regression,sentiment + ngram + type_dependency,0.779774
5,benevolent + hostile + other,benevolent + hostile + other,logistic_regression,sentiment + type_dependency,0.772004
6,benevolent + hostile + other,benevolent + hostile + other,logistic_regression,type_dependency,0.768605
7,benevolent + hostile + other,benevolent + hostile + other,svm,ngram,0.794544
8,benevolent + hostile + other,benevolent + hostile + other,svm,ngram + type_dependency,0.783147
9,benevolent + hostile + other,benevolent + hostile + other,svm,sentiment,0.591229


In [3]:
file_name='../results/results_rq1_20201120-182327.pkl'
#file_name='../results/results_rq1_20201123-093450.pkl'
#file_name='../results/results_rq1_20201123-122102.pkl'

In [4]:

import pickle
import pandas as pd

results_df=[]
with open(file_name, 'rb') as f:
    results_df = pickle.load(f)
    
results_df2=results_df[['train_domain', 'train_domain_modified', 'test_domain', 'test_domain_modified', 'model_name', 'feature_names', 'macro avg f1-score',]]
results_df2

results_df2.groupby(['model_name', 'train_domain_modified'])['macro avg f1-score'].mean().reset_index()

#df.groupby(['name', 'id', 'dept'])['total_sale'].mean().reset_index()

,model_name,train_domain_modified,macro avg f1-score
0,logistic_regression,False,0.739101
1,svm,False,0.757266


In [123]:
import pickle
import pandas as pd
pd.options.display.max_colwidth = 1200

results_df_all=[]
with open('../experiments/results_all.pkl', 'rb') as f:
    results_df_all = pickle.load(f)
    
def get_feature_names(x, ngram_range, ngram_range_td, rel):
    ll=[]
    for i in x:
        if i['name'] == 'ngram':
            reduced='reduced' if i['feature_selection'] == True else ''
            ll.append(' '.join((i['name'], ' words', str(ngram_range[0]), reduced)))
        elif i['name'] == 'type_dependency':
            reduced='reduced' if i['feature_selection'] == True else ''
            ll.append(' '.join(('ngram typed dependency', str(ngram_range_td[0]), reduced, rel)))
        else:
            ll.append(i['name'])
    return ' + '.join(ll)

def format_feature_names(r):
    ngram_range=''
    ngram_range_td=''
    rel=''
    
    if 'features__ngram__feature_extraction__ngram_range' in r['param_grid'].keys():
        ngram_range=r['param_grid']['features__ngram__feature_extraction__ngram_range']
    if 'features__type_dependency__feature_extraction__ngram_range' in r['param_grid'].keys():
        ngram_range_td=r['param_grid']['features__type_dependency__feature_extraction__ngram_range']
    if 'features__type_dependency__feature_extraction__type_dep_file_name' in r['param_grid'].keys():
        v=r['param_grid']['features__type_dependency__feature_extraction__type_dep_file_name'][0]
        rel=' (with relation)' if 'not_add_relation' in v else ' (without relation)'
    return get_feature_names(r['features'], ngram_range, ngram_range_td, rel)

def format_domain_name(r):
    dataset=r['dataset']
    modified='M' if r['modified'] == True else ''
    tr=''
    for d in dataset:
        tr=''.join((tr,d[0].upper()))
    return ''.join((tr,modified))

results_df_all.sort_values(by=['macro avg f1-score'], ascending=False)

df_group=results_df_all[(results_df_all['model_name'] != 'gender_word') &
                       (results_df_all['model_name'] != 'threshold_classifier')].groupby(
    ['train_domain','test_domain','model_name','features'])[
    ['macro avg f1-score', 'sexist precision','sexist recall', 
     'nonsexist precision','nonsexist recall']].mean().reset_index()
df_group.sort_values(by=['macro avg f1-score'], ascending=False)

,train_domain,test_domain,model_name,features,macro avg f1-score,sexist precision,sexist recall,nonsexist precision,nonsexist recall
14,BHO,BHO,logistic_regression,"ngram words (1, 3) + bert_doc",0.799995,0.802408,0.795920,0.798053,0.804140
52,BHO,BHO,logistic_regression,"sentiment + ngram words (1, 4) + ngram typed dependency (1, 1) (with relation) + bert_doc",0.799606,0.798770,0.801686,0.801541,0.797679
22,BHO,BHO,logistic_regression,"ngram words (1, 4) + ngram typed dependency (1, 1) (with relation) + bert_doc",0.799018,0.799589,0.798723,0.799582,0.799471
16,BHO,BHO,logistic_regression,"ngram words (1, 3) + ngram typed dependency (1, 1) (with relation) + bert_doc",0.798955,0.798676,0.800165,0.800244,0.797883
46,BHO,BHO,logistic_regression,"sentiment + ngram words (1, 3) + ngram typed dependency (1, 1) (with relation) + bert_doc",0.798784,0.799319,0.798409,0.799020,0.799266
...,...,...,...,...,...,...,...,...,...
91,BHO,BHO,svm,"sentiment + ngram words (1, 1) + bert_doc",0.762831,0.768882,0.757755,0.761728,0.768854
49,BHO,BHO,logistic_regression,"sentiment + ngram words (1, 4)",0.761894,0.733399,0.830602,0.804160,0.695857
108,BHO,BHO,svm,"sentiment + ngram words (1, 4)",0.761868,0.739741,0.816135,0.795284,0.709805
88,BHO,BHO,svm,sentiment,0.576232,0.569984,0.655406,0.592855,0.503434
